Installing ngrok

In [ ]:
!pip install pyngrok

In [ ]:
!ngrok authtoken

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install flask-ngrok

In [ ]:
cd /content/drive/MyDrive/handwrittenproject

/content/drive/MyDrive/handwrittenproject


In [ ]:
# import flask
# from flask import Flask,render_template,url_for,request
import pickle
import base64
import numpy as np
import cv2
import tensorflow as tf
import torch
from keras.models import load_model

In [ ]:
vect=None
# CLASSES = ["apple", "book", "bowtie", "candle", "cloud", "cup", "door", "envelope", "eyeglasses", "guitar", "hammer",
#            "hat", "ice cream", "leaf", "scissors", "star", "t-shirt", "pants", "lightning", "tree"]

emojis = [
    "Apple",
    "Bowtie",
    "Candle",
    "Door",
    "Envelope",
    "Fish",
    "Guitar",
    "Ice Cream",
    "Lightning",
    "Moon",
    "Mountain",
    "Star",
    "Tent",
    "Toothbrush",
    "Wristwatch"
]


In [ ]:
!pip install flask==0.12.2

In [ ]:
!pip install jinja2==3.0

In [ ]:
!pip install Werkzeug==2.2.2

In [ ]:
!pip install itsdangerous==2.0.1

In [ ]:
from flask import Flask,render_template,url_for,request

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template
from werkzeug.utils import secure_filename
from pyngrok import ngrok
import os

app = Flask(__name__)

# run_with_ngrok(app)

init_Base64 = 21;
# model = tf.keras.models.load_model('/content/drive/MyDrive/handwrittenproject/model/handwritten_digits.model')
# model = torch.load("/content/whole_model_quickdraw")
model = load_model('/content/QuickDraw.h5')

@app.route('/')
def home():
	return render_template('draw.html')

@app.route('/predict', methods=['POST'])
def predict():
  if request.method == 'POST':
          #Preprocess the image : set the image to 28x28 shape
          #Access the image
          draw = request.form['url']
          #Removing the useless part of the url.
          draw = draw[init_Base64:]
          #Decoding
          draw_decoded = base64.b64decode(draw)
          image = np.asarray(bytearray(draw_decoded), dtype="uint8")
          image = cv2.imdecode(image, cv2.IMREAD_GRAYSCALE)
          #Resizing and reshaping to keep the ratio.
          resized = cv2.resize(image, (28,28), interpolation = cv2.INTER_AREA)
          vect = np.array(resized, dtype=np.float32)
          # [None, None, :, :]
          # vect = torch.from_numpy(vect)
          # logits = model(vect)
          # print(CLASSES[torch.argmax(logits[0])])
          # vect = np.asarray(resized, dtype="uint8")
          # vect = vect.reshape(1, 1, 28, 28).astype('float32')
          vect = np.reshape(vect, (-1, 28, 28, 1))
          # vect = vect.reshape(-1, 28, 28)
          # processed = keras_process_image(image)
          # print("processed: " + str(processed.shape))
          pred_probab = model.predict(vect)[0]
          pred_class = list(pred_probab).index(max(pred_probab))
          index = emojis[pred_class]
          #Launch prediction
          # my_prediction = model.predict(vect)
          #Getting the index of the maximum prediction
          # index = np.argmax(my_prediction[0])
          # index = CLASSES[torch.argmax(logits[0])]
          #Associating the index and its value within the dictionnary

  return render_template('results.html', prediction =index)

# public_url = ngrok.connect(5000)

# if __name__ == '__main__':
#     app.run()

if __name__ == '__main__':
    # Set up the port
    port = 5003

    # Open an ngrok tunnel to the port
    public_url = ngrok.connect(port)
    print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")

    # Run the Flask app
    app.run(port=port)